<a href="https://colab.research.google.com/github/russellemergentai/FineTune_Run_MistralInstruct/blob/main/JackBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jack Bot, T4 is fine.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
!pip install -U bitsandbytes
!pip install torch transformers langchain
!pip install -U langchain-community
!pip install faiss-gpu

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

# load model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V5"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"
llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left', add_eos_token=True, use_fast=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
   ━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

In [5]:
# create retriever
from langchain.document_loaders import TextLoader
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, SequentialChain
from google.colab import output
from langchain import PromptTemplate

def create_retriever(path):
    if "https" in path:
      loader = WebBaseLoader(path)
    else:
      loader = TextLoader(path, encoding='UTF-8')

    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000, chunk_overlap=1000, separators=[" ", ",", "\n"]
    )

    texts = text_splitter.split_documents(documents)#
    model_path="intfloat/e5-large-unsupervised"

    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': False}
    )

    db = FAISS.from_documents(texts, embeddings)
    retriever = db.as_retriever()
    return retriever

# Define common pipeline parameters
common_params = {
    "torch_dtype": torch.bfloat16,
    "trust_remote_code": True,
    "device_map": {"": 0},
    "num_return_sequences": 1,
    "repetition_penalty": 1.5
}

# format output
def print_sections(result):
  words = result.split()
  for i in range(0, len(words), 12):
      chunk = words[i:i + 12]
      print(" ".join(chunk) + "\n")

# retrieve from data.txt or http page and run query
def retrieval_query_data():

  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>
   <s>
  """

  path="/content/drive/MyDrive/Target/data.txt"

  retriever = create_retriever(path)
  output.clear()

  query = input("enter your query on the uploaded data: ")

  pipelineQuery = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    max_length=2000,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    **common_params,
    max_new_tokens=512
  )

  llmPipelineQuery = HuggingFacePipeline(pipeline=pipelineQuery, model_kwargs={"temperature": 0.1})
  qa = RetrievalQA.from_chain_type(llm=llmPipelineQuery, retriever=retriever, return_source_documents=False)
  result = qa.run({"query": query})

  del pipelineQuery
  del llmPipelineQuery
  del qa
  import gc
  gc.collect()

  #print_sections(result)
  print(result)


def retrieval_query_web():

  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>
   <s>
  """

# use a web link i.e. # path="https://en.wikipedia.org/wiki/Footfall"
  path = input("enter http path: ")
  retriever = create_retriever(path)
  output.clear()

  query = input("enter your query on the uploaded data: ")

  pipelineQuery = pipeline(
      "text-generation",
      model=llm,
      tokenizer=tokenizer,
      #max_length=500,
      #do_sample=True,
      eos_token_id=tokenizer.eos_token_id,
      **common_params,
      max_new_tokens=100
  )

  llmPipelineQuery = HuggingFacePipeline(pipeline=pipelineQuery, model_kwargs={"temperature": 0.1})
  qa = RetrievalQA.from_chain_type(llm=llmPipelineQuery, retriever=retriever, return_source_documents=False)
  result = qa.run({"query": query})

  del pipelineQuery
  del llmPipelineQuery
  del qa
  import gc
  gc.collect()

  print_sections(result)


# summarize from data.txt
def summary_query():

  # Chain of summary
  summarize_template = """<s>[INST] The following is an article:
  {query}
    You will generate increasingly concise, entity-dense summaries of the above Article.

    Repeat the following 2 steps twice.
    Step 1. Identify 1-3 informative Entities ("; " delimited) from the Article which are missing from the previously generated summary.
    Step 2. Write a new, denser summary which covers every entity and detail from the previous summary plus the Missing Entities.

    A Missing Entity is:
    - Relevant: to the main article.
    - Specific: descriptive yet concise (5 words or fewer).
    - Non-redundant: not in the previous summary.

    Guidelines:
    - The first summary should be long (4-5 sentences, 100 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach 100 words.
    - Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
    - Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
    - The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
    - Missing entities can appear anywhere in the new summary.
    - Never drop entities from the previous summary. If space cannot be made, add fewer new entities.
  [/INST]
  </s>
  <s>
  """

  pipelineSUMMARY = pipeline(
      "text-generation",
      model=llm,
      tokenizer=tokenizer,
      max_length=2200,  # Increased to accommodate input + output tokens
      max_new_tokens=400,  # Limits the generated output to 200 tokens
      eos_token_id=tokenizer.eos_token_id,
      truncation=True,
      **common_params
  )

  llmPipelineSummary = HuggingFacePipeline(pipeline=pipelineSUMMARY, model_kwargs={"temperature": 0.05})

  path = "/content/drive/MyDrive/Target/data.txt"
  SUMMARIZE = PromptTemplate.from_template(summarize_template)
  llm_chain = SUMMARIZE | llmPipelineSummary

  loader = TextLoader(path, encoding = 'UTF-8')
  text = loader.load()

  result = llm_chain.invoke({"query": text})

  del pipelineSUMMARY
  del llmPipelineSummary
  del SUMMARIZE
  del llm_chain
  import gc
  gc.collect()

  print_sections(result)

# query the base model with a commentary from the fine tuned model
def standard_query():
  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>
   <s>
  """

  query = input("enter the llm query: ")

  # Initialize pipelines with specific parameters
  pipelineBase = pipeline(
      "text-generation",
      model=llm,
      tokenizer=tokenizer,
      #max_length=500,
      #do_sample=True,
      eos_token_id=tokenizer.eos_token_id,
      **common_params,
      max_new_tokens=200
  )

  # Create HuggingFacePipeline objects
  llmPipeline = HuggingFacePipeline(pipeline=pipelineBase, model_kwargs={"temperature": 0.1})
  PROMPT = PromptTemplate.from_template(prompt_template)
  llm_chain = PROMPT | llmPipeline
  result = llm_chain.invoke({"query": query})

  del pipelineBase
  del llmPipeline
  del PROMPT
  del llm_chain
  import gc
  gc.collect()

  print_sections(result)


In [6]:
# main loop
def main():
    functions = {'1': standard_query, '2': retrieval_query_data, '3': retrieval_query_web, '4': summary_query}

    while True:
        user_input = input("Enter 1 llm query, 2 data retrieval query, 3 web retrieval query, 4 data summary query or 'x' to exit: ").lower()
        output.clear()

        if user_input in functions:
            functions[user_input]()
        elif user_input == 'x':
            print("Exiting.")
            break
        else:
            print("Invalid input, please try again.")

if __name__ == "__main__":
    main()

RuntimeError: Error loading /content/drive/MyDrive/Target/data.txt